In [1]:
!pip install py2neo -q
!pip install neo4jupyter -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.0/177.0 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import os
import pandas as pd
import py2neo
from py2neo import Graph,Node,Relationship
import neo4jupyter
neo4jupyter.init_notebook_mode()
print ('py2neo version', py2neo.__version__)

<IPython.core.display.Javascript object>

py2neo version 2021.2.3


In [3]:
# Establish Neo4J connection Felipe
# dburl='neo4j+s://086d8743.databases.neo4j.io'
# user='neo4j'
# pasw='BxHc4q3iI30Gjkgc0PUbgEMD5hjzlbn6ENtGEGretNk'
# graph = Graph(dburl, auth=(user, pasw))

# Establish Neo4J connection Santiago
dburl='neo4j+s://59c0a233.databases.neo4j.io'
user='neo4j'
pasw='cVfbYqkChlNI32tHn2-Rv-gmJFL7a5LzKKohfP00utY'
graph = Graph(dburl, auth=(user, pasw))

# Establish your own Neo4J connection
# dburl=
# user=
# pasw=
# graph = Graph(dburl, auth=(user, pasw))

In [4]:
# Filtered files and attributes
# matches_csv='https://raw.githubusercontent.com/flacoski/tennis_data_BDNR/main/dataset_filtered/matches.csv'
# players_csv='https://raw.githubusercontent.com/flacoski/tennis_data_BDNR/main/dataset_filtered/players.csv'
# countries_csv='https://raw.githubusercontent.com/flacoski/tennis_data_BDNR/main/dataset_filtered/countries.csv'
# tourneis_csv='https://raw.githubusercontent.com/flacoski/tennis_data_BDNR/main/dataset_filtered/tournies.csv'
matches_csv='https://raw.githubusercontent.com/flacoski/tennis_data_BDNR/main/full_dataset/matches.csv'
players_csv='https://raw.githubusercontent.com/flacoski/tennis_data_BDNR/main/full_dataset/players.csv'
countries_csv='https://raw.githubusercontent.com/flacoski/tennis_data_BDNR/main/full_dataset/countries.csv'
tourneis_csv='https://raw.githubusercontent.com/flacoski/tennis_data_BDNR/main/full_dataset/tournies.csv'
matches_attributes = ['tourney_name','surface','draw_size',
                      'tourney_level','match_num','winner_id',
                      'winner_seed','winner_entry','winner_ht','winner_ioc',
                      'winner_age','loser_id','loser_seed','loser_entry',
                      'loser_ht','loser_ioc','loser_age','score','best_of',
                      'round','minutes','w_ace','w_df','w_svpt','w_1stIn',
                      'w_1stWon','w_2ndWon','w_SvGms','w_bpSaved','w_bpFaced',
                      'l_ace','l_df','l_svpt','l_1stIn','l_1stWon','l_2ndWon',
                      'l_SvGms','l_bpSaved','l_bpFaced','winner_rank',
                      'winner_rank_points','loser_rank','loser_rank_points',
                      'league']
player_attributes = ['player_id','name_first','name_last','hand','birthdate','country','gender']
player_node = ','.join([attr+":row."+attr for attr in player_attributes])
match_node = ','.join([attr+":row." + attr for attr in matches_attributes])+', tourney_date:date(row.tourney_date)'

In [6]:
# Load data
graph.run(f"LOAD CSV WITH HEADERS FROM '{countries_csv}' AS row CREATE (c:Country {{ country:row.Countries }})")
graph.run(f"LOAD CSV WITH HEADERS FROM '{tourneis_csv}' AS row CREATE (c:Tourney {{ name:row.tourney_name }})")
graph.run(f"""LOAD CSV WITH HEADERS FROM '{players_csv}' AS row
  CALL {{
    WITH row
    MATCH (c:Country {{country: row.country}})
    CREATE (p:Player {{ {player_node} }}) -[:BORN_IN]->(c)
  }} IN TRANSACTIONS OF 1000 ROWS
""")
graph.run(f"""CREATE INDEX player_id_index FOR (n:Player) ON (n.player_id)""")

graph.run(f"""
LOAD CSV WITH HEADERS FROM '{matches_csv}' AS row
 CALL {{
    WITH row
    MATCH (w:Player {{player_id:row.winner_id}}),
          (l:Player {{player_id:row.loser_id}}),
          (t:Tourney {{name:row.tourney_name}})
    CREATE (l)-[:LOST]->(n:Match {{ {match_node} }})<-[:WON]-(w),
           (n)-[:PART_OF]->(t)
 }} IN TRANSACTIONS OF 500 ROWS
 """)

(No data)

In [5]:
# Delete data - Solo ejecutar si se quiere borrar todos los datos

# graph.run("""
# MATCH (n)
# CALL {
#   WITH n
#   DETACH DELETE n
# } IN TRANSACTIONS OF 1000 ROWS
# """)

(No data)

In [7]:
# 4.1 Camino de menor longitud

graph.run("""
MATCH (p:Player {player_id:'103819'}), // Federer
(n:Player {player_id:'100029'}) // Rod Laver
RETURN shortestPath((p)-[:WON|LOST*]-(n))
""")

(No data)

In [ ]:
# 4.2 Duracion promedio

graph.run("""
MATCH (p:Player)-[:WON]->(m:Match)
WHERE m.minutes IS NOT NULL
RETURN p.player_id as id, p.name_first + ' ' + p.name_last as name, avg(toInteger(m.minutes)) AS minutes_avg, COUNT(m) AS matches, 1/(avg(toInteger(m.minutes))*sqrt(COUNT(m))) as weighted_avg
ORDER BY weighted_avg ASC
""")

In [ ]:
# 4.3 Ganador según superficie

graph.run("""
MATCH (c:Country)<-[:BORN_IN]-(p:Player)-[:WON]->(m:Match)
WITH m.surface AS surface, c.country AS country, COUNT(m) AS Wins
ORDER BY surface, Wins DESC
RETURN surface, COLLECT({country: country, wins: Wins})[0] AS maxWins
""")

In [ ]:
# 4.4 Ganador de torneo

graph.run("""
MATCH (t:Tourney)<-[:PART_OF]-(m:Match)<-[wl:WON|LOST]-(p:Player)
WITH t.name AS tourney_name, m.tourney_date AS tourney_date, p,
COUNT(CASE WHEN TYPE(wl) = 'WON' THEN 1 ELSE NULL END)
AS won_count
ORDER BY won_count DESC
WITH tourney_name, tourney_date,
COLLECT({player: p, won_count: won_count}) AS players
RETURN tourney_name, tourney_date,
players[0].player AS player,
players[0].won_count AS max_wins
""")

In [ ]:
# 4.5 Victorias transitivas

year = 2019

query = """
MATCH (pw:Player)-[w1:WON]->(m1:Match)<-[l1:LOST]-(pl:Player)-[w2:WON]->(m2:Match)<-[l2:LOST]-(pll:Player)
WHERE date({year:$year}) <= m1.tourney_date < date({year:$year+1})
AND date({year:$year}) <= m2.tourney_date < date({year:$year+1})
AND m2.tourney_date <= m1.tourney_date
AND pw.player_id <> pll.player_id
WITH pw, COLLECT(DISTINCT pl) + COLLECT(DISTINCT pll) AS distinctLostPlayers
RETURN pw.name_first + " " + pw.name_last as winner_name, SIZE(apoc.coll.toSet(distinctLostPlayers)) AS distinctCount
ORDER BY distinctCount DESC
"""

graph.run(query, year=year)

In [ ]:
# 4.6 Mejor racha de victorias

year_start = 2019
year_end = 2020

query = """
MATCH (pw:Player)-[r:WON|LOST]->(m1:Match)<-[:WON|LOST]->(pl:Player)
WHERE date({year:$year_start}) <= m1.tourney_date < date({year:$year_end})
WITH pw, m1, type(r) as results
ORDER BY m1.tourney_date ASC

// Obtener todas las rachas de victorias
WITH pw, REDUCE(s = [], result IN COLLECT(results) |
  CASE
    WHEN result = 'LOST' THEN s + [[]]
    ELSE s[0..size(s)-1] + [s[size(s)-1] + [result]]
  END
) AS all_winning_streaks

// Obtener todas la mejor racha de victorias
WITH pw, [arr IN all_winning_streaks WHERE size(arr) = REDUCE(maxSize = 0, a IN all_winning_streaks |
  CASE
    WHEN size(a) > maxSize THEN size(a)
    ELSE maxSize
  END
)][0] AS maximum_winning_streak

WHERE maximum_winning_streak IS NOT NULL
WITH pw, maximum_winning_streak, SIZE(maximum_winning_streak) AS streak_number
ORDER BY streak_number DESC
RETURN pw.name_first + ' ' + pw.name_last as player_name, streak_number
"""

graph.run(query, year_start=year_start, year_end=year_end)

